In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
from time import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

## dataset

In [5]:
df = pd.read_csv('./feature/df_feature2.csv', encoding='utf-8')

df.shape

(2100000, 27)

## train

In [6]:
from scipy.sparse import csr_matrix, csc_matrix, hstack

In [7]:
df = df.drop(['prefix', 'query_prediction', 'title', 'title_tag', 'query_prediction_origin'], axis=1)

In [8]:
cont_fea = ['dict_len', 'title_rate', 'max_title_rate', 'title_rate_max_sub', 'title_rank_in_query', 'title_rank_in_query_origin', 'rank_first_origin_rate', 'rank_first_origin_rate_sub']
# cont_fea += [i for i in df.columns if '_count' in i]
cate_fea = ['is_max_rate', 'is_first_rate', 'is_first_rate', 'title_equal_prefix', 'prefix_in_title', 'title_startswith_prefix','title_endswith_prefix', 'rate_big5', 'title_in_query']
long_cate_fea = ['user_id', 'item_id']

user_id_dict = dict(zip(df.user_id.unique(), range(len(df.user_id.unique()))))
item_id_dict = dict(zip(df.item_id.unique(), range(len(df.item_id.unique()))))

user_id_len = len(df.user_id.unique())
item_id_len = len(df.item_id.unique())

In [9]:
tag = pd.get_dummies(df.tag, prefix='tag')
cate_fea += list(tag.columns)
df = pd.concat([df, tag], axis=1)
df = df.drop(['tag'], axis=1)

df.shape

(2100000, 43)

In [10]:
ord_col = []
ord_row = list(range(0, df.shape[0]*2+2, 2))
ord_data = [1] * df.shape[0]*2

for index, row in tqdm_notebook(df[long_cate_fea].iterrows()):
    offset = 0
    for idx in long_cate_fea:
        ord_col.append(locals()[idx+'_dict'][row[idx]] + offset)
        offset += locals()[idx+'_len']
    
spr_df = csr_matrix((ord_data, ord_col, ord_row))

In [11]:
for i in tqdm_notebook(cate_fea):
    spr_tmp = csr_matrix(df[i]).transpose()
    spr_df = hstack((spr_df, spr_tmp))

In [13]:
from sklearn.preprocessing import MinMaxScaler
for fea in tqdm_notebook(cont_fea):
    scaler_val = df[fea][~df[fea].isnull()].values
    scaler = MinMaxScaler().fit(scaler_val.reshape((len(scaler_val), 1)))
    df[fea].fillna(df[fea].mean(), inplace=True)
    df[fea] = scaler.transform(df[fea].values.reshape((len(df), 1))).reshape((len(df),)).tolist()

In [23]:
cate_embedding_uni_cnt = {
    'user_id': 161916, 'item_id': 273686,'is_max_rate': 1,'is_first_rate': 1,'is_first_rate': 1,'title_equal_prefix': 1,
    'prefix_in_title': 1,'title_startswith_prefix': 1,'title_endswith_prefix': 1,'rate_big5': 1,'title_in_query': 1,'tag': 22}

cate_embedding_uni_cnt_list = [161916, 273686, 1, 1, 1, 1 ,1, 1, 1, 1, 1, 22]

In [33]:
y_train = df[df.is_val == 0].label
y_val = df[df.is_val == 1].label

In [48]:
from imp import reload
import model.dcfn.dcfn
reload(model.dcfn.dcfn)
from model.dcfn.dcfn import DCFN

<module 'model.dcfn.dcfn' from '/home/zero/oppo/model/dcfn/dcfn.py'>

In [49]:
print('loading conti data...')
# train_conti_feas, val_conti_feas = df[df.is_val == 0][cont_fea].as_matrix(), df[df.is_val == 1][cont_fea].as_matrix()
print('train conti feas shape: {}, val conti feas shape: {}'.format(np.shape(train_conti_feas), np.shape(val_conti_feas)))

print('loading ori cate data...')
# spr_df = spr_df.tocsr()
# train_cate_csr = spr_df[:2000000, :]
# val_cate_csr = spr_df[2000000:2050000, :]
print('train cate shape:{}, val cate shape:{}'.format(train_cate_csr.shape, val_cate_csr.shape))

print('training...')
model_name = 'oppo'
cate_embedding_w_list, fm_embedding_w, no_nan_w, nan_w = None, None, None, None

dcfn_params = {
    'learning_rate': 0.001,
    'embedding_size': 8,
    'dnn_layers': [2048, 512, 128],
    'att_layer': 8,
    'cross_layer_num': 1,
    'conti_fea_cnt': train_conti_feas.shape[1],
    'cate_embedding_uni_cnt_list': cate_embedding_uni_cnt_list,
    'cate_embedding_w_list': cate_embedding_w_list,
    'fm_embedding_w': fm_embedding_w,
    'no_nan_w': no_nan_w,
    'nan_w': nan_w,
    'fm_drop_outs': [0.5, 0.5],
    'result_weight': 0.5
}
model = DCFN(**dcfn_params)

fit_params = {
    'model_path': './model/nn/dcfm_%s.ckpt' % model_name,
    'batch_size': 1024,
    'epoch': 100,
    'cate_feas': train_cate_csr,
    'conti_feas': train_conti_feas,
    'labels': y_train.values.reshape(-1, 1),
    'v_cate_feas': val_cate_csr,
    'v_conti_feas': val_conti_feas,
    'v_labels': y_val.values.reshape(-1, 1),
    'es': 2
}
model.fit(**fit_params)

loading conti data...
train conti feas shape: (2000000, 8), val conti feas shape: (50000, 8)
loading ori cate data...
train cate shape:(2000000, 435633), val cate shape:(50000, 435633)
training...
embedding K:161916 -> 40
embedding K:273686 -> 45
embedding K:1 -> 1
embedding K:1 -> 1
embedding K:1 -> 1
embedding K:1 -> 1
embedding K:1 -> 1
embedding K:1 -> 1
embedding K:1 -> 1
embedding K:1 -> 1
embedding K:1 -> 1
embedding K:22 -> 4
y_deep:Tensor("Deep-part/LeakyRelu_2/Maximum:0", shape=(?, 128), dtype=float32),
 cross_output:Tensor("Cross-part/Reshape:0", shape=(?, 106), dtype=float32),
 y_first_order:Tensor("FM-part/dropout/mul:0", shape=(?, 106), dtype=float32),
 y_second_order:Tensor("FM-part/dropout_1/mul:0", shape=(?, 8), dtype=float32)
start training ---------------------------------------------------


ResourceExhaustedError: OOM when allocating tensor with shape[273686,45]
	 [[Node: cate_1_embedding/Adam/Assign = Assign[T=DT_FLOAT, _class=["loc:@cate_1_embedding"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](cate_1_embedding/Adam, cate_1_embedding/Adam/Initializer/zeros)]]

Caused by op 'cate_1_embedding/Adam/Assign', defined at:
  File "/usr/local/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/usr/local/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().